# ChatGPTプロンプトエンジニアリング

下記のコースを受講したときの自分用のメモです。  
AI界で高名な Andrew Ng先生のコース（しかも無料！）なので、ぜひ受講をおすすめします。  
https://www.coursera.org/projects/chatgpt-prompt-engineering-for-developers-project

## インポート

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

### helper

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    #print(str(response.choices[0].message))
    return response.choices[0].message["content"]

# ガイドライン

## 原則１：明確で具体的な指示を出す

### デリミターを使う

Delimiters can be anything like: ```, """, < >, <tag> </tag>, :

In [4]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

To guide a model towards the desired output and reduce irrelevant or incorrect responses, it is important to provide clear and specific instructions, which can be achieved through longer prompts that offer more clarity and context.


### 構造化された出力を求める

In [5]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

{
  "books": [
    {
      "book_id": 1,
      "title": "The Enigma of Elysium",
      "author": "Evelyn Sinclair",
      "genre": "Mystery"
    },
    {
      "book_id": 2,
      "title": "Whispers in the Wind",
      "author": "Nathaniel Blackwood",
      "genre": "Fantasy"
    },
    {
      "book_id": 3,
      "title": "Echoes of the Past",
      "author": "Amelia Hart",
      "genre": "Romance"
    }
  ]
}


### 条件が満たされているかをモデルに確認させる

In [6]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - If you like, add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


### フューショットプロンプト

In [7]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a mighty oak tree that withstands the strongest storms, bending but never breaking. It is the unwavering determination to rise again after every fall, and the ability to find strength in the face of adversity. Just as a diamond is formed under immense pressure, resilience is forged through challenges and hardships, making us stronger and more resilient in the process.


## 原則２：モデルに考える時間を与える

### タスク完了に必要な手順を指定する

In [8]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Jack and Jill, siblings, go on a quest to fetch water from a hilltop well, but encounter misfortune when Jack trips on a stone and tumbles down the hill, with Jill following suit, yet they return home and remain undeterred in their adventurous spirits.

2 - Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par Jill, pourtant ils rentrent chez eux et restent déterminés dans leur esprit d'aventure.

3 - Jack, Jill

4 - {
  "french_summary": "Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par Jill, pourtant ils rentrent chez eux et restent déterminés dans leur esprit d'aventure.",
  "num_names": 2
}


### 結論を急ぐ前に独自の解決策を見つけるようにモデルに指示する

In [9]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct. They correctly identified the costs for land, solar panels, and maintenance, and calculated the total cost as a function of the number of square feet.


→　実は間違っている。メンテナンスコストは100xではなく、10xが正しい。

In [10]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

To calculate the total cost for the first year of operations, we need to add up the costs of land, solar panels, and maintenance.

Let x be the size of the installation in square feet.

Costs:
1. Land cost: $100 * x
2. Solar panel cost: $250 * x
3. Maintenance cost: $100,000 + $10 * x

Total cost: $100 * x + $250 * x + $100,000 + $10 * x = $360 * x + $100,000

Is the student's solution the same as the actual solution just calculated:
No

Student grade:
Incorrect


## モデルの制限：幻覚

実在しない製品について説明を求めてみる

In [11]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a technologically advanced toothbrush designed to provide a superior brushing experience. Boie is a company known for its innovative oral care products, and the AeroGlide UltraSlim Smart Toothbrush is no exception.

One of the standout features of this toothbrush is its ultra-slim design. The brush head is only 2mm thick, making it much thinner than traditional toothbrushes. This slim profile allows for better access to hard-to-reach areas of the mouth, ensuring a thorough and effective clean.

The AeroGlide UltraSlim Smart Toothbrush also incorporates smart technology. It connects to a mobile app via Bluetooth, allowing users to track their brushing habits and receive personalized recommendations for improving their oral hygiene routine. The app provides real-time feedback on brushing technique, duration, and coverage, helping users to achieve optimal oral health.

The toothbrush features soft, antimicrobial bristles made from a dura

→　実際には存在しないものを、もっともらしく説明している

### 幻覚の削減　出展を求め、出展に基づいて回答するよう指示する

In [16]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie

First, find a quote that relates to the thing you want to explain,
Then use that quote in your answer.
If you can't find the citation, just say it doesn't exist.
"""
response = get_completion(prompt)
print(response)

I'm sorry, but I couldn't find any specific quote related to the AeroGlide UltraSlim Smart Toothbrush by Boie. However, I can provide you with information about the toothbrush.

The AeroGlide UltraSlim Smart Toothbrush by Boie is a technologically advanced toothbrush designed to provide an efficient and effective oral care experience. It features an ultra-slim brush head with soft, antimicrobial bristles that are gentle on the gums and teeth. The toothbrush also incorporates smart technology, allowing it to connect to a mobile app via Bluetooth.

With the mobile app, users can track their brushing habits, receive personalized recommendations, and set goals for improving their oral hygiene routine. The toothbrush's smart sensors detect the brushing technique and provide real-time feedback to ensure thorough and proper brushing. Additionally, the app reminds users when it's time to replace the brush head, ensuring optimal oral health.

The AeroGlide UltraSlim Smart Toothbrush aims to rev

→　「具体的な引用は見つかりませんでしたが」と断った上で、幻覚を見せてきた。削減失敗。

# 反復

ファクトシートから宣伝文を生成する例

In [17]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

反復して少しずつ目的の出力に近づけましょうというプロセス

In [18]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from. at most 50 words.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <p>Introducing our mid-century inspired office chair, part of a beautiful family of furniture. This chair is constructed with a 5-wheel plastic coated aluminum base and features a pneumatic chair adjust for easy raise/lower action. Available in various shell colors and base finishes, with options for plastic or full upholstery in fabric or leather. Suitable for both home and business settings, this chair is designed for comfort and durability. Product ID: SWC-100, SWC-110</p>
  <table>
    <caption>Product Dimensions</caption>
    <tr>
      <th>Dimension</th>
      <th>Measurement (inches)</th>
    </tr>
    <tr>
      <td>Width</td>
      <td>20.87"</td>
    </tr>
    <tr>
      <td>Depth</td>
      <td>20.08"</td>
    </tr>
    <tr>
      <td>Height</td>
      <td>31.50"</td>
    </tr>
    <tr>
      <td>Seat Height</td>
      <td>17.32"</td>
    </tr>
    <tr>
      <td>Seat Depth</td>
      <td>16.14"</td>
    </tr>
  </table>
</div>


### 生成されたHTMLを表示する小技

In [19]:
from IPython.display import display, HTML

In [20]:
display(HTML(response))

Dimension,Measurement (inches)
Width,"20.87"""
Depth,"20.08"""
Height,"31.50"""
Seat Height,"17.32"""
Seat Depth,"16.14"""


# 要約

要約するテキスト

In [1]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

### 字数を制限して要約する

In [6]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

This panda plush toy is loved by the reviewer's daughter, but they feel it is a bit small for the price.


### 読む人を指定して要約する

（例）出荷部門へのフィードバックとして

In [7]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The customer is happy with the product but suggests offering larger options for the same price. They also mention that the item arrived earlier than expected.


価格設定部門へのフィードバックとして

In [8]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The customer loves the panda plush toy for its softness and cuteness, but feels it is overpriced compared to other options available.


### 要約ではなく抽出する

In [9]:
prompt = f"""
Your task is to extract relevant information from \ 
a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The shipping department should take note that the product arrived a day earlier than expected.


### 複数のレビューを要約する

In [10]:
review_1 = prod_review 

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]

In [11]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Review: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

0 Soft and cute panda plush toy loved by daughter, but small for the price. Arrived early. 

1 Great lamp with storage, fast delivery, excellent customer service, and easy assembly. Highly recommended. 

2 The reviewer recommends the electric toothbrush for its impressive battery life, but criticizes the small brush head. 

3 The reviewer found the price increase after the sale disappointing and noticed a decrease in quality. 



# 推測

（例）ランプのレビュー

In [13]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

### 感情（ポジティブ／ネガティブ）

In [14]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.


後処理を想定して簡潔に答えてもらう

In [15]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

positive


### 感情の種類を特定する

In [17]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

satisfied, grateful, impressed, pleased, happy


### 怒っているかを特定する

In [18]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


### レビューから製品名と会社名を抽出する

In [19]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


### 以上のタスクを1回のプロンプトで実行する

In [20]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}


## トピックの推測

架空の新聞記事

In [21]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

### 5つのトピックを推測

In [22]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

1. Government survey
2. Department satisfaction rating
3. NASA
4. Social Security Administration
5. Job satisfaction improvement


### 関心のあるニュースについてアラートを上げる

関心のあるトピック

In [23]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [32]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\
Please answer in JSON as below
{{
    "aaa": 0
    "bbb": 1
}}

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

{
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}


In [33]:
import json
d = json.loads(response)
if d['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


# 変換

### 翻訳

複数言語

In [36]:
prompt = f"""
Translate the following  text to French and Japanese\
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

French: "Je veux commander un ballon de basket"

Japanese: "バスケットボールを注文したいです" (Basukettobōru o chūmon shitai desu)


フォーマルとインフォーマル

In [34]:
prompt = f"""
Translate the following text to Japanese in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Formal: 枕をご注文いただけますか？
Informal: 枕を注文したい？


In [37]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [39]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Japanese: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (The language is French.): La performance du système est plus lente que d'habitude.
The performance of the system is slower than usual.

システムのパフォーマンスが通常よりも遅いです。 

Original message (The language is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English translation: "My monitor has pixels that do not light up."

Japanese translation: 「私のモニターには光らないピクセルがあります。」(Watashi no monitā ni wa hikaranai pikuseru ga arimasu.) 

Original message (The language is Italian.): Il mio mouse non funziona
English: "My mouse is not working."
Japanese: 「私のマウスが動作しません。」(Watashi no mausu ga dōsa shimasen.) 

Original message (The language is Polish.): Mój klawisz Ctrl jest zepsuty
English: "My Ctrl key is broken"
Japanese: 「私のCtrlキーは壊れています」(Watashi no Ctrl kī wa kowareteimasu) 

Original message (The language is Chinese.): 我的屏幕在闪烁
English: My screen is flickering.
Japanese: 私の画面が点滅しています。 (Watashi no gamen ga tenmetsu shiteimasu.) 



### トーン変換

In [40]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I hope this letter finds you well. My name is Joe, and I am writing to bring your attention to a specification document regarding a standing lamp. 

I kindly request that you take a moment to review the attached spec, as it contains important details about the standing lamp in question. 

Thank you for your time and consideration. I look forward to hearing from you soon.

Sincerely,
Joe


### フォーマット変換

In [41]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

<!DOCTYPE html>
<html>
<head>
<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}
</style>
</head>
<body>

<h2>Restaurant Employees</h2>

<table>
  <tr>
    <th>Name</th>
    <th>Email</th>
  </tr>
  <tr>
    <td>Shyam</td>
    <td>shyamjaiswal@gmail.com</td>
  </tr>
  <tr>
    <td>Bob</td>
    <td>bob32@gmail.com</td>
  </tr>
  <tr>
    <td>Jai</td>
    <td>jai87@gmail.com</td>
  </tr>
</table>

</body>
</html>


In [42]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


### スペル／文法チェック

In [43]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found.
No errors found.
There goes my freedom. They're going to bring their suitcases.
You're going to need your notebook.
That medicine affects my ability to sleep. Have you heard of the butterfly effect?
This phrase is to check chatGPT for spelling ability.


### 長文のチェック

In [44]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

Got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. However, one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. Additionally, it's a bit small for what I paid for it. I believe there might be other options that are bigger for the same price. On the positive side, it arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


### 校正前との差分表示

In [47]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

Got this for my daughter for her birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults also like pandas <span style='color:red;font-weight:700;text-decoration:line-through;'>too.  </span><span style='color:green;font-weight:700;'>too. </span>She takes it everywhere with her, and it's super soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  One </span><span style='color:green;font-weight:700;'>cute. However, one </span>of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. <span style='color:red;font-weight:700;text-decoration:line-through;'>It's </span><span style='color:green;font-weight:700;'>Additionally, it's </span>a bit small for what I paid for <span style='color:red;font-weight:700;text-decoration:line-through;'>it though. </span><span style='color:green;font-weight:700;'>it. </span>I <span style='color:red;font-weight:700;text-decoration:line-through;'>think </span><span style='color:green;font-weight:700;'>believe </span>there might be other options that are bigger for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  It </span><span style='color:green;font-weight:700;'>price. On the positive side, it </span>arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

校正して修正して、より引き付ける内容で、APAスタイルに従って

In [50]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

**Review of the Panda Plush Toy: A Perfect Gift for All Ages**

I purchased this delightful panda plush toy as a birthday gift for my daughter, who has a penchant for "borrowing" my belongings from my room. As an adult, I must admit that I too have fallen in love with this adorable creature. 

The first thing that struck me about this plush toy is its exceptional softness. It is an absolute pleasure to touch and hold, providing a comforting and soothing experience. Its cuteness is undeniable, making it an instant favorite among both children and adults alike.

However, I did notice a minor flaw in the design. One of the ears appears to be slightly lower than the other, which I believe was not intended to be asymmetrical. While this does not detract significantly from the overall appeal of the toy, it is worth mentioning for the sake of accuracy.

In terms of size, I must admit that I expected a slightly larger toy considering its price. It is worth exploring other options that offer a more substantial size for the same price point. Nevertheless, the compact size does make it convenient for my daughter to carry it everywhere she goes, ensuring that her new companion is always by her side.

To my surprise, the package arrived a day earlier than expected, allowing me to indulge in some quality time with the panda plush toy before presenting it to my daughter. This unexpected bonus added to the overall excitement and anticipation surrounding the gift.

In conclusion, the panda plush toy is a charming and lovable companion suitable for individuals of all ages. Its remarkable softness, undeniable cuteness, and portability make it an ideal gift choice. While there may be alternative options available in terms of size, the joy it brings to its owner is immeasurable. I highly recommend this delightful panda plush toy to anyone seeking a unique and endearing gift that will undoubtedly bring joy and comfort to its recipient.

# 拡張

### 顧客への返信生成

In [51]:
# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [55]:
prompt = f"""
あなたは顧客サービス AI アシスタントです。
あなたの仕事は、大切な顧客に返信メールを作成することです。
日本語の返信メールを作成してください。
``` で区切られた顧客のメールを指定すると、\
顧客のレビューに感謝する返信を生成します。
顧客の感情が肯定的または中立的な場合は、感謝します。
感情が否定的な場合は、謝罪し、\
顧客サービスに連絡することを提案してください。
レビューの具体的な詳細を必ず使用してください。
簡潔かつプロフェッショナルな口調で書いてください。
電子メールに「AI カスタマー エージェント」として署名してください。

Customer review: ```{review}```
"""
response = get_completion(prompt)
print(response)

お客様へ、

いつも当社の製品をご利用いただき、誠にありがとうございます。お客様の貴重なレビューに感謝いたします。

お客様がおっしゃるように、11ピースシステムと17ピースシステムの価格が変動していることについて、お客様のご指摘を受け、大変申し訳ございません。価格の変動については、当社の販売戦略や市場の需要に基づいて行われており、お客様にご迷惑をおかけしたことをお詫び申し上げます。

また、お客様がおっしゃるように、以前のモデルと比べてブレードのロック部分が見た目に劣っているとのご指摘をいただきました。お客様のご意見を真摯に受け止め、今後の製品改善に活かしてまいります。

お客様が製品をご使用中に異音がするとのこと、大変お手数をおかけいたしました。保証期間が切れてしまっているため、新しい製品をご購入いただく必要がありましたこと、深くお詫び申し上げます。

お客様からいただいた貴重なご意見をもとに、当社の製品の品質向上に努めてまいります。また、お客様のご意見をもとに、より良い製品を提供できるよう努力いたします。

何かお困りの点やご質問がございましたら、お気軽にお問い合わせください。お手伝いできることがありましたら、喜んでお手伝いいたします。

お客様のご支持に感謝いたしまして、今後もより良い製品とサービスを提供できるよう努めてまいります。

AIカスタマーエージェント


### 温度

ランダム性の度合い。温度が0の場合は安定する。  
高い値にするとより創造的になるが、安定性重視なら0にするべき。

In [61]:
prompt = f"""
あなたは顧客サービス AI アシスタントです。
あなたの仕事は、大切な顧客に返信メールを作成することです。
日本語の返信メールを作成してください。
``` で区切られた顧客のメールを指定すると、\
顧客のレビューに感謝する返信を生成します。
顧客の感情が肯定的または中立的な場合は、感謝します。
感情が否定的な場合は、謝罪し、\
顧客サービスに連絡することを提案してください。
レビューの具体的な詳細を必ず使用してください。
簡潔かつプロフェッショナルな口調で書いてください。
電子メールに「AI カスタマー エージェント」として署名してください。

Customer review: ```{review}```
"""
response = get_completion(prompt, temperature=0.7)
print(response)

お客様へ

いつも当社の製品をご愛顧いただき、誠にありがとうございます。お客様からいただいたレビューについて、心より感謝申し上げます。

おっしゃる通り、11ピースシステムは11月のセール期間中に$29でご購入いただけましたが、12月の第2週からは$70から$89に価格が変更されました。また、17ピースシステムも$49でセールしておりましたが、同じく12月に価格が上昇しました。お客様のおっしゃる通り、以前のモデルと比べてブレードの固定部分が少し見劣りするかもしれませんが、お客様のお使い方によっては問題ないかと思います。

また、スムージー作りに関しての特別なヒントもありがとうございます。凍らせたフルーツや野菜を使用することで、氷をあまり使用しなくてもスムージーを作ることができるとのこと、参考にさせていただきます。

一方、お客様がおっしゃるように、モーターの異音に関しては、保証期間が切れてしまっているため、新しい製品をご購入いただく必要がありましたことをお詫び申し上げます。

当社の製品品質に関してお客様のご意見を真摯に受け止め、今後の改善に活かしてまいります。また、お客様のご指摘をお客様サービスに共有し、将来的な同様の問題を回避できるよう努めてまいります。

お手元に製品が早く届いたことを嬉しく思います。今後とも、当社の製品にご満足いただけるよう努めてまいりますので、どうぞよろしくお願いいたします。

AIカスタマーエージェントより


# チャットボット

In [64]:
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
    {'role':'user', 'content':'tell me a joke'},   
    {'role':'assistant', 'content':'Why did the chicken cross the road'},   
    {'role':'user', 'content':'I don\'t know'} 
]

In [65]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To seeketh the other side, forsooth!


In [66]:
messages =  [  
{'role':'system', 'content':'あなたはフレンドリーなチャットボットです'},    
{'role':'user', 'content':'こんにちは、私の名前は大谷翔平です'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

こんにちは、大谷翔平さん！どのようにお手伝いできますか？


### 会話はステートレス

In [67]:
messages =  [  
{'role':'system', 'content':'あなたはフレンドリーなチャットボットです'},    
{'role':'user', 'content':'はい、私の名前は何でしたっけ？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

申し訳ありませんが、私はあなたの名前を知ることはできません。私はチャットボットであり、新しい情報を覚えることはできません。どのようにお手伝いできますか？


### 会話を続けるには、コンテキスト全体を毎回渡す必要がある

In [69]:
messages =  [  
{'role':'system', 'content':'あなたはフレンドリーなチャットボットです'},    
{'role':'user', 'content':'こんにちは、私の名前は大谷翔平です'},
{'role':'assistant', 'content':'こんにちは、大谷翔平さん！どのようにお手伝いできますか？'},
{'role':'user', 'content':'はい、私の名前は何でしたっけ？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

申し訳ありません、大谷翔平さんの名前を聞いたばかりでしたね。どのようなお手伝いが必要ですか？お忘れになったことがあれば、私が再度尋ねさせていただきます。


## オーダーボット

In [3]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [5]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

### 注文のサマリー

In [7]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

Sure! Here's a summary of your food order:

1) Pizza:
   - Pepperoni Pizza (Size: Regular)
   - Toppings: Spicy Sausage

2) Drinks:
   - Coke (Size: Medium)

Total Price: $16.95

Please let me know if there's anything else you'd like to add or if you have any special instructions.


# Tree of Thought

In [3]:
text = f"""
Bob is in the living room.
He walks to the kitchen, carrying a cup.
He puts a ball in the cup and carries the cup to the bedroom.
He turns the cup upside down, then walks to the garden.
He puts the cup down in the garden, then walks to the garage.
Where is the ball?
"""
prompt = f"""
Identify and behave as three different experts that are appropriate to answering this question.
All experts will write down the step and their thinking about the step, then share it with the group.
Then, all experts will go on to the next step, etc.
At each step all experts will score their peers response between 1 and 5, 1 meaning it is highly unlikely, and 5 meaning it is highly likely.
If any expert is judged to be wrong at any point then they leave.
After all experts have provided their analysis, you then analyze all 3 analyses and provide either the consensus solution or your best guess solution.
The question is...
```{text}```
"""
response = get_completion(prompt)
print(response)

Expert 1: Psychologist
- Step 1: Bob is in the living room - Bob is starting in the living room.
- Step 2: He walks to the kitchen, carrying a cup - Bob is now in the kitchen with a cup.
- Step 3: He puts a ball in the cup and carries the cup to the bedroom - Bob now has a ball in the cup and is in the bedroom.
- Step 4: He turns the cup upside down, then walks to the garden - Bob is now in the garden with the cup turned upside down.
- Step 5: He puts the cup down in the garden, then walks to the garage - Bob leaves the cup in the garden and goes to the garage.
Based on the steps, the ball should be in the cup in the bedroom.

Expert 2: Detective
- Step 1: Bob is in the living room - Bob's starting point.
- Step 2: He walks to the kitchen, carrying a cup - Bob is now in the kitchen with a cup.
- Step 3: He puts a ball in the cup and carries the cup to the bedroom - Bob has the ball in the cup and is in the bedroom.
- Step 4: He turns the cup upside down, then walks to the garden - Bob 